<a href="https://colab.research.google.com/github/aiforpeople/Natural-Language-Processing-NLP-/blob/master/Classification_of_Wine%2Cfrom_reviews_with_SVC%2CNaive_Bayes_Deep_Learning(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To build a deep learning algorithm to determine the variety of the wine being reviewed based on the review text.The wine magazine dataset at https://www.kaggle.com/zynicide/wine-reviews which is provided by Kaggle user zackthoutt is used for the study

Conceptually, the question is, can we take a wine review like…

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn’t overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.""

…and recognize that it is about a white blend? Some wine enthusiasts might recognize telltale signs of white wines such as apple, citrus, and a pronounced acidity, but can we train our neural network to recognize these signals? Additionally, can we train our neural network to recognize the subtle differences between a white blend review and a pinot grigio review?

**Similar Algorithms**
The problem is essentially an NLP classification problem. There are several NLP classification algorithms that have been applied to various problems in NLP. For example, naive Bayes have been used in various spam detection algorithms, and support vector machines (SVM) have been used to classify texts such as progress notes at healthcare institutions. It would be interesting to implement a simple version of these algorithms to serve as a baseline for thisdeep learning model.

**Naive Bayes**
A popular implementation of naive Bayes for NLP involves preprocessing the text using TF-IDF and then running the multinomial naive Bayes on the preprocessed outputs. This allows the algorithm to be run on the most prominent words within a document.The naive Bayes is implemented as follows:

In [0]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer



In [0]:
df = pd.read_csv('winemag-data-130k-v2.csv')

In [0]:
df.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [0]:
print(type(df['variety']))
counter = Counter(df['variety'].tolist())
print(type(df['variety']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [0]:
counter=Counter(df['variety'].tolist())
counter

Counter({'Abouriou': 3,
         'Agiorgitiko': 63,
         'Aglianico': 359,
         'Aidani': 1,
         'Airen': 3,
         'Albana': 23,
         'Albanello': 1,
         'Albariño': 477,
         'Albarossa': 2,
         'Aleatico': 5,
         'Alfrocheiro': 16,
         'Alicante': 5,
         'Alicante Bouschet': 50,
         'Aligoté': 30,
         'Alsace white blend': 51,
         'Altesse': 8,
         'Alvarelhão': 2,
         'Alvarinho': 137,
         'Alvarinho-Chardonnay': 5,
         'Ansonica': 2,
         'Antão Vaz': 17,
         'Apple': 2,
         'Aragonez': 10,
         'Aragonês': 10,
         'Argaman': 3,
         'Arinto': 77,
         'Arneis': 86,
         'Asprinio': 3,
         'Assyrtico': 43,
         'Assyrtiko': 18,
         'Athiri': 1,
         'Austrian Red Blend': 62,
         'Austrian white blend': 80,
         'Auxerrois': 27,
         'Avesso': 6,
         'Azal': 3,
         'Babić': 4,
         'Babosa Negro': 1,
         'Bacchus': 2

In [0]:
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
top_10_varieties

{'Bordeaux-style Red Blend': 4,
 'Cabernet Sauvignon': 2,
 'Chardonnay': 1,
 'Merlot': 9,
 'Pinot Noir': 0,
 'Red Blend': 3,
 'Riesling': 5,
 'Rosé': 8,
 'Sauvignon Blanc': 6,
 'Syrah': 7}

In [0]:
df=df[df['variety'].map(lambda x: x in top_10_varieties)]

In [0]:
df.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
12,12,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
14,14,US,Building on 150 years and six generations of w...,NaN,87,12.0,California,Central Coast,Central Coast,Matt Kettmann,@mattkettmann,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou


In [0]:
description_list = df['description'].tolist()
print(type(df['description']))
print(type(description_list))

<class 'pandas.core.series.Series'>
<class 'list'>


In [0]:
description_list[0:3]

['Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
 "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",
 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.']

In [0]:
print(description_list[0:3],sep=':')

['Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.', "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.", 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.']


In [0]:
varietal_list = [top_10_varieties[i] for i in df['variety'].tolist()]
varietal_list = np.array(varietal_list)

In [0]:
varietal_list

array([5, 0, 2, ..., 2, 5, 0])

In [0]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)

In [0]:
x_train_counts.shape

(71322, 23888)

In [0]:
x_train_counts[0:3]

<3x23888 sparse matrix of type '<class 'numpy.int64'>'
	with 99 stored elements in Compressed Sparse Row format>

In [0]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [0]:
x_train_tfidf[0:3]

<3x23888 sparse matrix of type '<class 'numpy.float64'>'
	with 99 stored elements in Compressed Sparse Row format>

In [0]:
train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

In [0]:
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

In [0]:
print(n_right)
print(len(test_y))

13574
21397


In [0]:
print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 63.44%


Considering  10 classes, this is quite a good result.

Use support vector machine and see how it would do. To see how it performs, simply replace the classifier definition with

In [0]:
from sklearn import svm
clf = svm.SVC(kernel='linear').fit(train_x, train_y)
y_score = clf.predict(test_x)
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print(" SVC Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

 SVC Accuracy: 80.60%


In [0]:
counter = Counter(df['variety'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['variety'].map(lambda x: x in top_10_varieties)]

description_list = df['description'].tolist()
varietal_list = [top_10_varieties[i] for i in df['variety'].tolist()]
varietal_list = np.array(varietal_list)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 63.26%


Deep Learning with NLP

**Building the Model**
Today, we will be using Keras with Tensorflow to build our model. Keras is a Python library that makes building deep learning models very easy compared to the relatively low-level interface of the Tensorflow API. In addition to the dense layers, we will also use embedding and convolutional layers to learn the underlying semantic information of the words and potential structural patterns within the data.

**Data Cleaning**
It is essential to restructure the data in a way that can be easily processed and understood by the neural network. Replacing the words with uniquely identifying numbers helps.Combined with an embedding vector, the words can be represented  in a manner that is both flexible and semantically sensitive.

In preprocessing phase It would make sense to focus on the commonly used words, and to also filter out the most commonly used words (e.g., the, this, a).

Implement this functionality using Defaultdict and NLTK.( Write the following code into a separate Python module)

In [0]:
from nltk import word_tokenize
from collections import defaultdict

def count_top_x_words(corpus, top_x, skip_top_n):
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]


def replace_top_x_words_with_vectors(corpus, top_x):
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict


def filter_to_top_x(corpus, n_top, skip_n_top=0):
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

Now it is time to build the model. The network will have an  embedding layer, a convolutional layer, and a dense layer to take advantage of all of the deep learning features that can be helpful for theapplication. The model can be built with Keras.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from lib.get_top_xwords import filter_to_top_x

df = pd.read_csv('data/wine_data.csv')

counter = Counter(df['variety'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['variety'].map(lambda x: x in top_10_varieties)]

description_list = df['description'].tolist()
mapped_list, word_list = filter_to_top_x(description_list, 2500, 10)
varietal_list_o = [top_10_varieties[i] for i in df['variety'].tolist()]
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from lib.get_top_xwords import filter_to_top_x

df = pd.read_csv('data/wine_data.csv')

counter = Counter(df['variety'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['variety'].map(lambda x: x in top_10_varieties)]

description_list = df['description'].tolist()
mapped_list, word_list = filter_to_top_x(description_list, 2500, 10)
varietal_list_o = [top_10_varieties[i] for i in df['variety'].tolist()]
varietal_list = to_categorical(varietal_list_o)

max_review_length = 150

mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)
train_x, test_x, train_y, test_y = train_test_split(mapped_list, varietal_list, test_size=0.3)

max_review_length = 150

embedding_vector_length = 64
model = Sequential()

model.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(max(varietal_list_o) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=3, batch_size=64)

y_score = model.predict(test_x)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

In [0]:

#from lib.get_top_xwords import filter_to_top_x
import nltk
nltk.download('punkt')


description_list = df['description'].tolist()
mapped_list, word_list = filter_to_top_x(description_list, 2500, 10)
varietal_list_o = [top_10_varieties[i] for i in df['variety'].tolist()]
varietal_list = to_categorical(varietal_list_o)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
max_review_length = 150

mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)
train_x, test_x, train_y, test_y = train_test_split(mapped_list, varietal_list, test_size=0.3)

max_review_length = 150

embedding_vector_length = 64

In [0]:
model = Sequential()

model.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(max(varietal_list_o) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=3, batch_size=64)

Epoch 1/3
49925/49925 [==============================] - 44s 877us/step - loss: 1.0095 - acc: 0.6473
Epoch 2/3
49925/49925 [==============================] - 43s 864us/step - loss: 0.6270 - acc: 0.7854
Epoch 3/3
49925/49925 [==============================] - 43s 867us/step - loss: 0.5217 - acc: 0.8210


In [0]:
y_score = model.predict(test_x)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 77.49%


Build a model that was able to compete with and outperform some of the other machine learning algorithms.